In [1]:
from PIL import Image
import cv2
import msvcrt
import smtplib
import numpy as np
from io import BytesIO
from pytesseract import Output
from email.message import EmailMessage
import datetime
import os
from selenium import webdriver
from scipy.ndimage.filters import rank_filter
from time import sleep
from pytesseract import pytesseract
import requests
from bs4 import BeautifulSoup as bs
import config

In [2]:
url = "https://www.mintscan.io/sentinel/validators/sentvaloper1tt53juuu8g2ux67vgqzu9lkhfn9w6z73x4rfrk"

In [7]:
'''
This loop will continuously run, if the Uptown falls below a threshold a single email wil be sent to a recipient.
Once uptime goes back to 100% the status will reset to 0 and a subsequent fall will advise the user of the activity
again. The output could easily be sent to a continously growing database for EDA.
'''

status = 0

while True:
    
    try:

        driver = webdriver.Chrome('./chromedriver')

        driver.get(url)
        sleep(10)

        png = driver.get_screenshot_as_png()

        driver.quit()

        im = Image.open(BytesIO(png))

        # im = im.crop((left, top, right, bottom))
        im = im.crop((40, 450, 1100, 725))
        im.save("Stats.png")

        test_im = cv2.imread("Stats.png")
        
        # Uncomment to see cropped images for tesseract processing
        #cv2.imshow('Recover', test_im)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()

        path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
        pytesseract.tesseract_cmd = path_to_tesseract

        text_list = []
        extract = pytesseract.image_to_string(test_im)
        text_list = extract.split("\n")
        text_list = [i for i in text_list if i]
        ct = datetime.datetime.now()
        text_dict = {'Time': ct.strftime('%Y-%m-%d::%H-%M'),text_list[0]:text_list[6], text_list[1]:text_list[7], text_list[2]:text_list[8], text_list[3]:text_list[9], text_list[4]:text_list[10], text_list[5]:text_list[11]}
        print(text_dict)

        uptime = text_dict['Uptime']
        uptime = float(uptime.strip("%"))
        #uptime = 70

        if uptime < 90 and status == 0:
            status = 1
            msg = EmailMessage()
            msg.set_content('Uptime is less than 90')

            msg['Subject'] = 'Uptime Status Check'
            msg['From'] = config.MY_EMAIL
            msg['To'] = config.MY_EMAIL # Enter a gmail, yahoo, etc address here as a str

            s= smtplib.SMTP_SSL('smtp.gmail.com', 465)
            s.login(config.MY_EMAIL, config.MY_PASS)
            s.send_message(msg)
            s.quit()
            
        elif uptime == 100:
            status = 0
            
        else:
            continue

    except KeyError:
        print("Could not retrieve Data from: ", url, "!Check your url")

C:\Users\jab39\AppData\Local\Temp/ipykernel_752/3049175428.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


{'Time': '2022-05-31::16-32', 'Website': 'https://www.dweb.net/', 'Commission': '5.00%', 'Uptime': '100%', 'Voting Power': '0.28% (31,850,808.404439 DVPN)', 'Bonded Height': '4,125,892', 'Self Bonded': '0.149985 DVPN (0.00%)'}
{'Time': '2022-05-31::16-33', 'Website': 'https://www.dweb.net/', 'Commission': '5.00%', 'Uptime': '100%', 'Voting Power': '0.28% (31,850,808.404439 DVPN)', 'Bonded Height': '4,125,892', 'Self Bonded': '0.149985 DVPN (0.00%)'}
{'Time': '2022-05-31::16-33', 'Website': 'https://www.dweb.net/', 'Commission': '5.00%', 'Uptime': '100%', 'Voting Power': '0.28% (31,850,808.404439 DVPN)', 'Bonded Height': '4,125,892', 'Self Bonded': '0.149985 DVPN (0.00%)'}
{'Time': '2022-05-31::16-33', 'Website': 'https://www.dweb.net/', 'Commission': '5.00%', 'Uptime': '100%', 'Voting Power': '0.28% (31,850,808.404439 DVPN)', 'Bonded Height': '4,125,892', 'Self Bonded': '0.149985 DVPN (0.00%)'}
{'Time': '2022-05-31::16-34', 'Website': 'https://www.dweb.net/', 'Commission': '5.00%', 'Up

KeyboardInterrupt: 